In [16]:
from datetime import datetime
from faker import Faker
import pandas as pd
from tqdm.notebook import tqdm

In [17]:
social_media_users_df = pd.read_csv("./genres/genres.csv")

In [18]:
genres = set()

for _, row in tqdm(social_media_users_df[["Interests"]].iterrows(), total=len(social_media_users_df)):
    for genre in row["Interests"].replace("'", "").split(","):
        genre = genre.strip()
        genres.add(genre)

  0%|          | 0/100000 [00:00<?, ?it/s]

In [19]:
fake = Faker()

In [20]:
def create_fake_person():
    birth_date = fake.date_of_birth()
    age = datetime.date(datetime.now()).year - birth_date.year
    
    bot_score = fake.pyfloat(left_digits=0, right_digits=2, positive=True, min_value=None, max_value=None)
  
    _, _, city, country, state = fake.local_latlng(country_code="US")
    state = state.split("/")[-1].replace("_", " ")
    location = " ".join([city, state, country])
  
    return [fake.uuid4(), fake.first_name(), fake.last_name(), birth_date.strftime("%Y-%m-%d"), age, location, fake.random_element(genres), bot_score]

In [21]:
people = [create_fake_person() for _ in range(1000)]

In [22]:
def create_fake_poster():
    birth_date = fake.date_of_birth()
    age = datetime.date(datetime.now()).year - birth_date.year
    
    return [fake.uuid4(), fake.user_name(), age, ";".join([fake.random_element(genres) for _ in range(fake.random_int(min=1, max=5))])]

In [23]:
posters = [create_fake_poster() for _ in range(200)]

In [24]:
followers = []

for person in people:
    followers.extend([[person[0], fake.random_element(people)[0]] for _ in range(fake.random_int(min=2, max=10))])

In [25]:
subscribers = []

for person in people:
    subscribers.extend([[person[0], fake.random_element(posters)[0]] for _ in range(fake.random_int(min=2, max=10))])

In [26]:
people_df = pd.DataFrame(people, columns=["personId:ID(Person-ID)", "first_name", "last_name", "birth_date:date", "age:int", "location", "genre", "bot_score:float"])
people_df[":LABEL"] = "person"
people_df.head()

,personId:ID(Person-ID),first_name,last_name,birth_date:date,age:int,location,genre,bot_score:float,:LABEL
0,4c419279-d0d6-4290-9563-b42db18dbe9b,Mark,Fields,1981-11-05,43,Brentwood Estates Chicago US,Music,0.12,person
1,a7d0de70-bd31-45df-8df5-189966f1fe2e,Christine,Murphy,2005-03-12,19,Kensington New York US,Technology,0.77,person
2,62b2066b-5fbb-44b3-8899-0aa44f528ff8,Theresa,Frye,2012-02-21,12,Keene New York US,Technology,0.76,person
3,9bef669c-7e66-4d86-bb82-ad3be9ac7c31,Jennifer,Rice,2007-03-27,17,Murray Chicago US,Business and entrepreneurship,0.27,person
4,e2aed787-255e-463b-a4a0-b1944ee8c706,Sarah,Anderson,1925-10-31,99,Palm Desert Los Angeles US,Photography,0.57,person


In [27]:
posters_df = pd.DataFrame(posters, columns=["posterId:ID(Poster-ID)", "username", "age:int", "genres:string[]"])
posters_df[":LABEL"] = "poster"
posters_df.head()

,posterId:ID(Poster-ID),username,age:int,genres:string[],:LABEL
0,425e1eb3-8805-49ab-bfef-152d91488e9e,wlowe,74,Cars and automobiles;Pets;Finance and investme...,poster
1,b17c5a97-fce9-4d76-b5b9-d8a3bcccef13,bmccoy,109,Politics;DIY and crafts,poster
2,4644c19b-254d-4024-a1ef-0fa9dc4c53a1,hollywalker,88,Travel;History;Travel;Cooking,poster
3,99974735-183f-4a87-83c2-793915cf3ab0,deborah95,108,Gaming,poster
4,b5f18a49-8ac0-41af-a678-bfabf85025ad,kcampbell,7,Business and entrepreneurship;Cars and automob...,poster


In [28]:
followers_df = pd.DataFrame(followers, columns=["personId:START_ID(Person-ID)", "personId:END_ID(Person-ID)"])
followers_df[":TYPE"] = "follows"
followers_df.head()

,personId:START_ID(Person-ID),personId:END_ID(Person-ID),:TYPE
0,4c419279-d0d6-4290-9563-b42db18dbe9b,aa147d2e-dae2-4818-a250-ce29120b300e,follows
1,4c419279-d0d6-4290-9563-b42db18dbe9b,d59e676a-4e09-4531-a8a7-51d4c0fb7822,follows
2,4c419279-d0d6-4290-9563-b42db18dbe9b,00e32a49-782a-486a-8a73-2d39059a33a0,follows
3,4c419279-d0d6-4290-9563-b42db18dbe9b,028229be-e884-4e9e-87c9-aeea00a7ce9b,follows
4,4c419279-d0d6-4290-9563-b42db18dbe9b,90d08060-d006-4060-9ac4-9b8074c5a02f,follows


In [29]:
subscribers_df = pd.DataFrame(subscribers, columns=["personId:START_ID(Person-ID)", "posterId:END_ID(Poster-ID)"])
subscribers_df[":TYPE"] = "subscribes_to"
subscribers_df.head()

,personId:START_ID(Person-ID),posterId:END_ID(Poster-ID),:TYPE
0,4c419279-d0d6-4290-9563-b42db18dbe9b,32b71e1c-15ca-4a24-9005-8162a3bd2154,subscribes_to
1,4c419279-d0d6-4290-9563-b42db18dbe9b,83e5ce32-9637-4b15-8565-8e22465209fc,subscribes_to
2,4c419279-d0d6-4290-9563-b42db18dbe9b,f48d8d5b-5a4b-4aff-8d52-f5f1e7517ba4,subscribes_to
3,a7d0de70-bd31-45df-8df5-189966f1fe2e,77e95c60-3fb8-4214-b636-930f2f589339,subscribes_to
4,a7d0de70-bd31-45df-8df5-189966f1fe2e,9de005e4-0426-4f48-a248-722a6d9ca84c,subscribes_to


In [30]:
people_df.to_csv("./fake-social-media-data/nodes/people.csv", index=False)
posters_df.to_csv("./fake-social-media-data/nodes/posters.csv", index=False)
followers_df.to_csv("./fake-social-media-data/edges/follows.csv", index=False)
subscribers_df.to_csv("./fake-social-media-data/edges/subscribes.csv", index=False)